In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *
from numpy import *

# import JSON module for storing and loading data
import json

# you need to run 'pip install progressbar2' for this to work
import progressbar

In [2]:
m = UNITS.meter
s = UNITS.second
kg = UNITS.kilogram
degree = UNITS.degree
N = UNITS.newton

<Unit('newton')>

In [3]:
default_params = Params(
    x = 0 * m,
    y = 10000 * m,
    speed = 88.889 * m/s,
    mass = 900 * kg,
    g = 3.7 * m/s**2,
    diam = 3 * m,
    C_d = 0.3,
    rho = 1.2 * 0.006 * kg/m**3,
    t_end = 3600 * s,
    angle = -45 * degree,
    Thrust = 3300 * kg * m/s**2
)

,values
x,0 meter
y,10000 meter
speed,88.889 meter / second
mass,900 kilogram
g,3.7 meter / second ** 2
diam,3 meter
C_d,0.3
rho,0.0072 kilogram / meter ** 3
t_end,3600 second
angle,-45 degree


In [4]:
def make_system(params):
    """ Make a system object.
    
    params: Parameter object includes
     - x: initial x position (m)
     - y: initial y position (m)
     - speed: initial speed (m/s)
     
     - mass: rover mass (kg)
     - g: acceleration due to gravity (m/s^2)
     
     - diam: rover diameter (m)
     - C_d: coefficient of drag (unitless)
     - rho: air density at surface level (kg/m^3)
     
     - t_end: end time (s)
     
     - angle: entry angle (degree)
     - Thrust: thrust force (N)
    
    returns: System object includes
     - mass: rover mass (kg)
     - area: rover surface area (m^2)
     - g: acceleration due to gravity (m/s^2)
     - thrust: acceleration due to thrust (m/s^2)
     - C_d: coefficient of drag (unitless)
     - rho: air density at surface level (kg/m^3)
     - t_end: end time (s)
     - init: State object includes
      - x
      - y
      - vx
      - vy
    """
    
    unpack(params)
    
    area = np.pi * (diam/2)**2
    
    thrust = Thrust / mass
    
    theta = np.deg2rad(angle)
    vx, vy = Vector(*pol2cart(theta, speed))
    init = State(x=x, y=y, vx=vx, vy=vy)
    
    return System(
        mass = mass,
        area = area,
        g = g,
        thrust = thrust,
        C_d = C_d,
        rho = rho,
        t_end = t_end,
        init = init
    )

In [5]:
make_system(default_params)

,values
mass,900 kilogram
area,7.0685834705770345 meter ** 2
g,3.7 meter / second ** 2
thrust,3.6666666666666665 meter / second ** 2
C_d,0.3
rho,0.0072 kilogram / meter ** 3
t_end,3600 second
init,x 0 meter y ...


In [6]:
def drag_force(state, system):
    """Computes drag force in the opposite direction of velocity.
    
    state: State object with pos, v
    system: System object with area, C_d, rho
    
    returns: Vector drag force
    """
    
    x, y, vx, vy = state
    pos = Vector(x,y)
    v = Vector(vx,vy)
    unpack(system)
    
    C_a = np.exp(-(pos.y/1000)/np.e)
    mag = -(rho * C_a) * v.mag**2 * C_d * area / 2
    direction = v.hat()
    f_drag = mag * direction
    return f_drag

In [7]:
def slope_func(state, t, system):
    """Computes derivatives of the state variables.
    
    state: State (x, y, vx, vy)
    t: time
    system: System object with g, rho, C_d, area, mass
    
    returns: sequence (v.x, v.y, a.x, a.y)
    """
    x, y, vx, vy = state
    pos = Vector(x,y)
    v = Vector(vx,vy)
    unpack(system)
  
    a_drag = drag_force(state, system) / mass
    a_grav = Vector(0, -g)
    a_thrust = Vector(0, thrust)
    
    a = a_grav + a_thrust + a_drag
    
    return v.x, v.y, a.x, a.y

In [8]:
def event_func(state, t, system):
    """Stop when the y coordinate is 0.
    S
    state: State object
    t: time
    system: System object
    
    returns: y coordinate
    """
    x, y, vx, vy = state
    pos = Vector(x,y)
    v = Vector(vx,vy)
    
    return pos.y

In [9]:
def landing_speed(Thrust, params):
    """Computes the speed at which the rover hits the ground for a given thrust.
    
    Thrust: thrust force in kg * m/s^2
    params: Params object
    
    returns: landing speed in m/s
    """
    params = Params(params, Thrust=Thrust * kg * m/s**2)
    system = make_system(params)
    results, details = run_ode_solver(system, slope_func, events=event_func)
    final_v = Vector(get_last_value(results.vx), get_last_value(results.vy))
    final_speed = final_v.mag * m/s
    return final_speed

In [10]:
def thrust_for_angle(angle, params):
    """Computes the optimal thrust for a given angle.
    
    angle: entry angle in degrees
    params: Params object
    
    returns: thrust in kg * m/s^2
    """

    new_params = Params(params, angle=angle * degree)
    result = min_bounded(landing_speed, [0,10000], new_params)
    optimal_thrust = result.x
    return optimal_thrust

In [11]:
def optimize_all_angles(speed, params, **kwargs):
    """Computes optimal thrust, impact speed, and time for all 
    angles between 0 and -90 degrees given entry speed.
    
    speed: entry speed in m/s
    params: Params object
    progress: (optional) progress bar object to update
    count: (optional) counter variable for tracking progress
    
    returns: Dictionary object includes
     - key: angle in degrees
      - optimal_thrust: optimal thrust in kg * m/s^2
      - impact_speed: landing speed in m/s
      - time: descent time in s
    """
    if 'progress' in kwargs and 'count' in kwargs:
        progress = kwargs['progress']
        count = kwargs['count']
    else:
        progress = None
        count = None
    
    params = Params(params, speed=speed * m/s)
    
    sweep_data = {}
    
    for angle in range(-90, 1):
        params = Params(params, angle=angle * degree)

        opt_result = min_bounded(landing_speed, [3000,5500], params)
        optimal_thrust = opt_result.x
        impact_speed = opt_result.fun
        
        params = Params(params, Thrust=optimal_thrust)
        system = make_system(params)
        sim_results, sim_details = run_ode_solver(system, slope_func, events=event_func)
        time = get_last_value(sim_results.index)
        
        sweep_data[angle] = {
            'optimal_thrust': optimal_thrust,
            'impact_speed': impact_speed.item(),
            'time': time
        }
        
        if progress:
            count += 1
            progress.update(count)
    
    return sweep_data

In [12]:
def sweep_speeds_and_export(speeds, params):
    """For each speed, this function records the optimum thrust, landing speed, 
    and descent time for every entry angle between 0 and -90 degrees.
    It saves all the data for each speed as an individual JSON file.
    
    speeds: list of all speeds to test
    params: Params object
    
    returns: None
    """
    
    bar_widgets = [
        progressbar.Percentage(),
        ' | ',
        progressbar.SimpleProgress(),
        ' | ',
        progressbar.ETA()
    ]
    
    progress_count = 0
    
    with progressbar.ProgressBar(max_value=len(speeds) * 91, widgets=bar_widgets) as bar:
        for idx, speed in enumerate(speeds):
            data = optimize_all_angles(speed, params, progress=bar, count=idx*91)
            filepath = f'data/{speed}.json'
            with open(filepath, 'w') as outfile:
                json.dump(data, outfile)
    
    return

In [13]:
sweep_speeds_and_export(range(10,201), default_params)

100% | 17381 of 17381 | ETA:  00:00:00                                         


NameError: name 'results' is not defined